[View in Colaboratory](https://colab.research.google.com/github/meghamattikalli/internship/blob/master/final_ann_25_epoch.ipynb)

In [3]:
from google.colab import files
uploaded = files.upload()

Saving five_thousand.csv to five_thousand.csv
Saving Indian-Female-Names1.csv to Indian-Female-Names1.csv
Saving Indian-Male-Names1.csv to Indian-Male-Names1.csv


0.9178749921917916


In [7]:
# -*- coding: utf-8 -*-
"""
Created on Tue Aug 14 12:45:30 2018
Applying K-fold cross validation
3 hidden layers with 100,50,10 nodes respectively
50 epochs
acc- 91.34
@author: HP
"""

# -*- coding: utf-8 -*-
"""
Created on Tue Aug 14 11:53:53 2018
accuracy is 0.889
roc-auc is 0.889
3 hidden layers
Names with less than 2 letters removed
10000 samples

@author: HP
"""

import pandas as pd
import io

female = pd.read_csv(io.StringIO(uploaded['Indian-Female-Names1.csv'].decode('utf-8')))

male = pd.read_csv(io.StringIO(uploaded['Indian-Male-Names1.csv'].decode('utf-8')))

eng_word = pd.read_csv(io.StringIO(uploaded['five_thousand.csv'].decode('utf-8')))


# Importing the dataset
#female= pd.read_csv('Indian-Female-Names1.csv')
female= female.apply(lambda x: x.astype(str).str.lower())
female=female.drop_duplicates(subset=['word'], keep='first')
female=female.iloc[:,0:1]
female["type"]=1

unwanted=['mr','.','moh.','@','miss','&','mrs','/','ku.','km',',','km0','-','(','smt','smt.','`','[','ms']
for i in unwanted:
    female['word'] = female.word.str.replace(i, '')
numbers=['1','2','3','4','5','6','7','8','9','0']
for i in numbers:
    female['word'] = female.word.str.replace(i, '')
female['word'] = female.word.str.strip()
female['word'] = female.word.str.partition(" ")
female=female.drop_duplicates(subset=['word'], keep='first')

#preprocessing of names
#female['word'] = female.word.str.replace('miss', '')
#female['word'] = female.word.str.replace('mrs', '') 
#female['word'] = female.word.str.replace('.', '')
#female['word'] = female.word.str.replace('smt', '')
#female['word'] = female.word.str.replace('@', '')
#female['word'] = female.word.str.strip()
#female['word'] = female.word.str.partition(" ")
#female=female.drop_duplicates(subset=['word'], keep='first')
female=female[female['word'].map(len) > 2]
female=female.head(2500)


#male= pd.read_csv('Indian-Male-Names1.csv')
male= male.apply(lambda x: x.astype(str).str.lower())
male= male.drop_duplicates(subset=['word'], keep='first')
male=male.iloc[:,0:1]
male["type"]=1

for i in unwanted:
    male['word'] = male.word.str.replace(i, '')
for i in numbers:
    male['word'] = male.word.str.replace(i, '')
    
male['word'] = male.word.str.strip()
male['word'] = male.word.str.partition(" ")
male=male.drop_duplicates(subset=['word'], keep='first')

#male['word'] = male.word.str.replace('mr', '')
#male['word'] = male.word.str.replace('.', '')
#male['name'] = male.name.str.replace('smt', '')
#male['word'] = male.word.str.replace('@', '')
#male['word'] = male.word.str.strip()
#male['word'] = male.word.str.partition(" ")
male=male[male['word'].map(len) > 2]
male=male.head(2500)


#english words
#eng_word = pd.read_csv('five_thousand.csv')
eng_word = eng_word.apply(lambda x: x.astype(str).str.lower())
eng_word['word'] = eng_word.word.str.replace('-', '')
eng_word['word'] = eng_word.word.str.replace("'", '')
eng_word['word'] = eng_word.word.str.replace("/", ' ')
eng_word['word'] = eng_word.word.str.partition(" ")


#eng_pre = pd.read_csv('prepositions.csv')
eng_word['type'] = 0
#eng_pre['type'] = 0


#concatenate names and english words
names=pd.concat([female,male,eng_word], axis=0)
names=names.dropna()


names.index=[i for i in range(0,len(names.index))] #renaming the indices

X=names['word'] #features
y=names['type'] #labels
        
#padding and truncating X
for i in range(0,len(X.index)):
    if(len(X.loc[i])>10):
        X.loc[i]=X.loc[i][0:10]
    else:    
        X.loc[i]=X.loc[i].ljust(10) 

def letter_to_int(letter):
    alphabet = list('abcdefghijklmnopqrstuvwxyz ')
    return alphabet.index(letter)


list1=[]
for j in range(0,len(X.index)):
    z=[]
    for i in X.loc[j]:
        z.append(letter_to_int(i))
    list1.append(tuple(z))
        
df=pd.DataFrame(list1)

from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(categorical_features = [0])
df1 = onehotencoder.fit_transform(df).toarray()
df1 = df1[:, 1:]

for i in range(9,0,-1):
    zzz=len(df1[0])-i
    onehotencoder = OneHotEncoder(categorical_features = [zzz])
    df1 = onehotencoder.fit_transform(df1).toarray()
    df1 = df1[:, 1:]
    

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1, y, test_size = 0.2, random_state = 0)


# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 100, kernel_initializer = 'uniform', activation = 'relu', input_dim =len(df1[0])))
    classifier.add(Dense(units = 50, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 25)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 5, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print(mean)
print(variance)





/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Epoch 1/25
Epoch 1/25
6400/6400 [==============================] - 11s 2ms/step - loss: 0.4419 - acc: 0.7773
Epoch 2/25
6400/6400 [==============================] - 11s 2ms/step - loss: 0.4338 - acc: 0.7848
Epoch 2/25
6400/6400 [==============================] - 9s 1ms/step - loss: 0.3076 - acc: 0.8667
Epoch 3/25
6400/6400 [==============================] - 9s 1ms/step - loss: 0.2933 - acc: 0.8755
Epoch 3/25
6400/6400 [==============================] - 9s 1ms/step - loss: 0.2426 - acc: 0.8994
Epoch 4/25
6400/6400 [==============================] - 9s 1ms/step - loss: 0.2236 - acc: 0.9080
Epoch 4/25
6400/6400 [==============================] - 9s 1ms/step - loss: 0.1850 - acc: 0.9269
Epoch 5/25
6400/6400 [==============================] - 9s 1ms/step - loss: 0.1705 - acc: 0.9325
Epoch 5/25
6400/6400 [==============================] - 9s 1ms/step - loss: 0.1457 - acc: 0.9453
Epoch 6/25
6400/6400 [==============================] - 10s 2ms/step - loss: 0.1342 - acc: 0.9489
Epoch 6/25
6400/